In [1]:
import numpy as np
import pandas as pd

In [2]:
pl_df = pd.read_csv('this_one_problem_logs_seq.csv')

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (21,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
pl_df['end_time'].head()

0    2009-03-18 10:44:07.266522
1    2009-03-18 10:50:23.111935
2    2009-03-18 10:51:24.231159
3    2009-04-29 11:00:52.685771
4    2009-05-11 15:47:47.425749
Name: end_time, dtype: object

In [3]:
# read 9 experiments
problem_set = 263052
ps_df = pd.read_csv('~/git/22-Experiments/'+str(problem_set)+'_exp.csv', header=None)

In [4]:
print len(ps_df[1].unique())
print len(ps_df[ps_df[2]==1])
print len(ps_df[ps_df[2]==0])

382
184
198


In [5]:
train_df = pl_df[pl_df['user_id'].isin(ps_df[1].unique().tolist()) & (pl_df['original'] == 1)].reset_index()

In [6]:
print(len(train_df['user_id'].unique()))

928


In [7]:
# read ps_list
ps_list = []
with open(str(problem_set)+'_ps_list') as file:
    for line in file:
        line = line.strip()
        ps_list.append(int(line))

In [8]:
len(ps_list)

583

In [9]:
encode_list = []
for index, row in train_df.iterrows():
    encode = np.zeros(len(ps_list)+1)
    try:
        idx = ps_list.index(row['sequence_id'])
        encode[idx] = 1
    except ValueError:
        encode[len(ps_list)] = 1

    encode_list.append(encode)
encode_array = np.stack(encode_list, axis=0)

In [14]:
final_df = pd.DataFrame(encode_array, columns=ps_list+['-'], dtype=int)
# u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time
final_df['correct'] = np.where(train_df['correct'] < 1, 0, 1)
final_df['bottom_hint'] = train_df['bottom_hint']
final_df['hint_count'] = train_df['hint_count']
final_df['attempt_count'] = train_df['attempt_count']
final_df['first_response_time'] = train_df['first_response_time']
# first action
final_df['first_action'] = train_df['first_action']

norm = lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() - x.min() else x/(x+1)

final_df.insert(len(final_df.columns), 'NofFRT', train_df.groupby('sequence_id')['first_response_time'].transform(norm))
del final_df['first_response_time']

final_df.insert(len(final_df.columns), 'normalized_hint_count', train_df.groupby('sequence_id')['hint_count'].transform(norm))
del final_df['hint_count']

final_df.insert(len(final_df.columns), 'normalized_attempt_count', train_df.groupby('sequence_id')['attempt_count'].transform(norm))
del final_df['attempt_count']

final_df['user_id'] = train_df['user_id']
final_df['id'] = train_df['id']
final_df['NofFRT'].fillna(0.5, inplace=True)
final_df.isnull().values.any()

False

In [15]:
final_df.to_csv(str(problem_set)+'_train_data.csv', index=False)